In [1]:
import sys
sys.path.append('CONFIG_files/')

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import shutil

from matplotlib import pyplot as plt
from tabulate import tabulate

import os, shutil, time, pdb, random
import scipy.stats as stats 
import scipy

from math import pi
from datetime import datetime
from collections import OrderedDict
import pickle

import torch
from torch.utils.data import TensorDataset, DataLoader

from importlib import reload
from models import *
from utils import *
from runmanager import *
from experiment import *
from plot_utils import *
from preprocessing_utils import *
from analysis_seasonal import * 

from sklearn.metrics import mean_squared_error as mse

import matplotlib
matplotlib.rc_file_defaults()
%matplotlib inline

# import CONFIG_SUSHIWAT_BGMM_23_JUNE_2023 as C
import CONFIG as C

pd.options.display.max_columns = None

np.random.seed(4)

%load_ext autoreload
%autoreload 2

device = C.device

C.REGION, C.RUN_NAME, C.split_by

('SUSHIWAT', 'SUSHIWAT_BGMM_09_OCT_2023_EXP_1', 'station')

### Import data

In [3]:
data = DataPreprocessing(train_path=C.TRAIN_PATH, start=C.start, end=C.end, 
                add_yesterday = False, 
                basin_filter = None, 
                split_bias_corrected_only = False, 
                filter_incomplete_years = False, 
                include_non_bc_stations = True, 
                split_by = C.split_by)

### Split data into held out sets for K-fold cross validation

In [4]:
SORT_BY_ELEVATION = False

if SORT_BY_ELEVATION:
    data.split_stations(sort_by_elev = True)
else:
    data.split_stations(sort_by_elev = False)

### Prepare data

In [5]:
# C.predictors.append('obs_yesterday')
data.input_data(C.predictors, C.predictand, sort_by_quantile=C.sort_by_quantile)
print(C.sort_by_quantile)

False


## Multi-Run: Train model with different hyperparameters

### Model run

In [6]:
experiment_label = C.RUN_NAME

In [7]:
experiment_label, C.K_FOLD

('SUSHIWAT_BGMM_09_OCT_2023_EXP_1', 10)

In [8]:
load_run = None

In [9]:
if load_run == None:
    src = 'CONFIG.py'
    dst = f'CONFIG_files/CONFIG_{experiment_label}.py'
    shutil.copy(src, dst)

st_test, predictions, importances = multirun(data, C.predictors, C.params, C.epochs, 
                                split_by = 'station', sequential_samples = True, n_samples=1, 
                                load_run = load_run, #None, 
                                experiment_label = experiment_label, feature_attribution = False, 
                                show_loss_plot=False)

,run,epoch,loss,valid_loss,test_loss,decision_loss,epoch duration,run duration,lr,batch_size,likelihood_fn,dropout_rate,random_noise,k,model_arch
0,1,1,1.706470,1.542926,1.772155,1.542926,43.474975,43.505986,0.001,32,bgmm,0,0,0,"(SimpleRNN, [10])"
1,1,2,1.650723,1.543304,1.773700,1.543304,43.685493,87.265645,0.001,32,bgmm,0,0,0,"(SimpleRNN, [10])"
2,1,3,1.648394,1.539983,1.775499,1.539983,39.346869,126.675210,0.001,32,bgmm,0,0,0,"(SimpleRNN, [10])"
3,1,4,1.649567,1.544402,1.775243,1.544402,39.017154,165.751792,0.001,32,bgmm,0,0,0,"(SimpleRNN, [10])"
4,1,5,1.650870,1.542836,1.773305,1.542836,39.302563,205.114011,0.001,32,bgmm,0,0,0,"(SimpleRNN, [10])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10,6,1.653992,1.577579,1.747935,1.577579,40.443414,242.083750,0.001,32,bgmm,0,0,9,"(SimpleRNN, [10])"
96,10,7,1.653186,1.580762,1.748931,1.580762,40.565939,282.705860,0.001,32,bgmm,0,0,9,"(SimpleRNN, [10])"
97,10,8,1.653215,1.580853,1.750732,1.580853,40.399656,323.180671,0.001,32,bgmm,0,0,9,"(SimpleRNN, [10])"
98,10,9,1.652705,1.586369,1.749132,1.586369,39.774901,363.017573,0.001,32,bgmm,0,0,9,"(SimpleRNN, [10])"


0
10000
20000
SimpleRNN_[10]_bgmm_B=32_D=0_RN=0


/users/marron31/repos/upper-indus-prec-bc/utils.py:1102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions[run]['k_all'] = predictions[run]['k_all'].append(predictions[run][f'k{i}'])
/users/marron31/repos/upper-indus-prec-bc/utils.py:1102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions[run]['k_all'] = predictions[run]['k_all'].append(predictions[run][f'k{i}'])
/users/marron31/repos/upper-indus-prec-bc/utils.py:1102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions[run]['k_all'] = predictions[run]['k_all'].append(predictions[run][f'k{i}'])
/users/marron31/repos/upper-indus-prec-bc/utils.py:1102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versio

In [14]:
# table_of_predictions_ks_test(predictions, C.seasons, columns, sample_cols, add_cols)

In [17]:
# table_of_predictions_for_metric(predictions, C.seasons, columns, C.n_samples, sample_cols, add_cols, metric = 'smape', prefix='smape')

In [18]:
# table_of_predictions_for_metric(predictions, C.seasons, columns, C.n_samples, sample_cols, add_cols, metric = 'edd', prefix='edd')

In [19]:
# table_of_predictions_for_metric(predictions, C.seasons, columns, C.n_samples, sample_cols, add_cols, metric = 'ae', prefix='ae')

In [ ]:
a = pd.read_csv('results.csv')

b = a.groupby(['k','run']).agg({'valid_loss': 'min', 
                                 'model_arch': 'first', 
                                 'likelihood_fn': 'first',
                                 'lr':'first',
                                 'batch_size':'first',
                                 'dropout_rate':'first',
                               })

c = b.groupby(['run']).agg({'valid_loss': 'mean', 
                        'model_arch': 'first', 
                        'likelihood_fn': 'first',
                        'lr':'first',
                        'batch_size':'first',
                        'dropout_rate':'first',
                         }
                   ).sort_values('valid_loss').reset_index()

c.groupby(['model_arch',
           'likelihood_fn',
           'lr',
           'batch_size',
           'dropout_rate',
           ]).agg({'valid_loss': 'mean'}
                   ).sort_values('valid_loss').reset_index()